In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras import applications
from keras.optimizers import SGD,Adam
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
ls

AV GOT.ipynb  sample_submission_ns2btKE.csv  train.zip
result1.csv   test_ApKoW4T.csv               X_train.npy
result2.csv   train/                         y_train.npy


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='1'

In [3]:
# loading data
with open("X_train.npy","rb") as fp:
    X = np.load(fp)
with open("y_train.npy","rb") as fp:
    y = np.load(fp)

In [5]:
type = ['none','Cargo', 
'Military', 
'Carrier', 
'Cruise', 
'Tankers']

# model definition

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [6]:
def res_models(img_rows, img_cols, channel=1, num_classes=None):
    model = applications.resnet50.ResNet50(include_top=True, weights='imagenet')

    # Truncate and replace softmax layer for transfer learning
    
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    
    new_model = Sequential()
    new_model.add(model)
    new_model.add(Dense(num_classes, activation='softmax'))

#     Uncomment below to set the first 10 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:5]:
        layer.trainable = True

    # Learning rate is changed to 0.001
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return new_model

In [7]:
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 6
batch_size = 1
nb_epoch = 10

In [8]:
model = res_models(img_rows, img_cols, channel, num_classes)

In [9]:
for i in model.layers:
    print(i,i.trainable)

<keras.engine.training.Model object at 0x7fd7cebd2710> True
<keras.layers.core.Dense object at 0x7fd7cdb082b0> True


In [10]:
# Start Fine-tuning
model.fit(X, y,
          nb_epoch=10,
          shuffle=True,
          verbose=1,
          validation_split=0.1
          )

/home/humonics/.virtualenvs/capi/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 5626 samples, validate on 626 samples
Epoch 1/10
5626/5626 [==============================] - 48s 9ms/step - loss: 0.5305 - acc: 0.8011 - val_loss: 0.2872 - val_acc: 0.8930
Epoch 2/10
5626/5626 [==============================] - 45s 8ms/step - loss: 0.0999 - acc: 0.9714 - val_loss: 0.3037 - val_acc: 0.8898
Epoch 3/10
5626/5626 [==============================] - 46s 8ms/step - loss: 0.0302 - acc: 0.9954 - val_loss: 0.2647 - val_acc: 0.9153
Epoch 4/10
5626/5626 [==============================] - 46s 8ms/step - loss: 0.0164 - acc: 0.9979 - val_loss: 0.2770 - val_acc: 0.9169
Epoch 5/10
5626/5626 [==============================] - 47s 8ms/step - loss: 0.0103 - acc: 0.9989 - val_loss: 0.2851 - val_acc: 0.9249
Epoch 6/10
5626/5626 [==============================] - 46s 8ms/step - loss: 0.0064 - acc: 0.9995 - val_loss: 0.2996 - val_acc: 0.9217
Epoch 7/10
5626/5626 [==============================] - 46s 8ms/step - loss: 0.0046 - acc: 1.0000 - val_loss: 0.3101 - val_acc: 0.9137
Epoch 8/

In [44]:
model.save_weights('ResNet50.hdf5')
print("model saved..!")

model saved..!


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12294     
Total params: 23,600,006
Trainable params: 23,546,886
Non-trainable params: 53,120
_________________________________________________________________


In [14]:
ls

'AV GOT.ipynb'*   sample_submission_ns2btKE.csv*   train1.h5      y_train.npy*
 my_model.h5      test_ApKoW4T.csv*                train.zip*
 my_model.hdf5    train/                           X_train.npy*


In [15]:
test = pd.read_csv('test_ApKoW4T.csv')

In [16]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('train/images/'+test['image'][i], target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 2680/2680 [00:03<00:00, 782.78it/s]


In [17]:
feature = model.predict(test)

In [25]:
results = []
for i in range(feature.shape[0]):
    pred = np.argmax(feature[i])
    results.append(pred)

In [40]:
len(results)

2680

In [36]:
test = pd.read_csv('test_ApKoW4T.csv')

## Generating Submission

In [39]:
sample = pd.read_csv('sample_submission_ns2btKE.csv')
sample['image'] = test['image']
sample['category'] = results
sample.to_csv('result1.csv', header=True, index=False)

### saving to .npy

In [31]:
train = pd.read_csv('train/train.csv')

In [32]:
train.head(5)

,image,category
0,2823080.jpg,1
1,2870024.jpg,1
2,2662125.jpg,2
3,2900420.jpg,3
4,2804883.jpg,2


In [ ]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/images/'+train['image'][i], target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

In [ ]:
!rm -f X_train.npy

In [ ]:
ls

In [ ]:
np.save('X_train.npy', X)

In [ ]:
y=train['category'].values
y = to_categorical(y)

In [ ]:
np.save('y_train.npy', y)

In [1]:
pwd

'/data2/charan/api/AV/GOT DeepLearning'

## Ensemble

In [1]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os

Using TensorFlow backend.
